## Tools with MCP

Model Context Protocol (MCP) provides a standardized way to connect AI agents to external tools and data sources. Let's connect to an MCP server using `langchain-mcp-adapters`

In [8]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# connect to the mcp-time server for timezone aware operations
# This GO based server provides tools for current time, relative time parsing,
# timezone conversion, duration arithmetic and time comparison
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@theo.foobar/mcp-time"],
        }
    }
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[tool.name for tool in mcp_tools]}")

Loaded 5 MCP tools: ['add_time', 'compare_time', 'convert_timezone', 'current_time', 'relative_time']


/tmp/ipykernel_110980/733504994.py:20: RuntimeWarning: coroutine 'MultiServerMCPClient.get_tools' was never awaited
  mcp_tools = await mcp_client.get_tools()


### Create an agent with tools (mcp_tools)

In [15]:
from langchain.agents import create_agent

agent = create_agent(
    model="ollama:gpt-oss:20b-cloud",
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

In [16]:
result = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "What is the current time in Tokyo?"}]}
)
print(result["messages"][-1].content)

The current time in Tokyo is **2025‑11‑03 18:27:17 +09:00**.
